In [ ]:
import pandas as pd

Document search with embeddings

In [ ]:

%pip install -U -q "google-genai>=1.0.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 5.4 MB/s eta 0:00:00


In [ ]:
from IPython.display import Markdown

In [ ]:
from google import  genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:

from google.genai import types
title = "Prompt engineering"
sample_text = """
title: Prompt engineering
 Author: Lee Boonstra
 """
EMBEDDING_MODEL_ID = "models/embedding-001"
embedding = client.models.embed_content(
        model=EMBEDDING_MODEL_ID,
        contents=sample_text,
        config=types.EmbedContentConfig(
            task_type="retrieval_document",
            title=title
    ))

print(embedding)


embeddings=[ContentEmbedding(values=[0.032113794, -0.039372865, -0.0039141006, -0.044029947, 0.062632635, 0.049092975, 0.029433437, -0.025812684, 0.022853756, 0.03743845, 0.012629882, -0.0025197107, -0.03484893, 0.007882097, -0.012547428, -0.036580723, 0.05770092, 0.024394179, -0.01136753, -0.008917674, 0.010298141, 0.023751942, 0.0054585487, -0.028781576, 0.043187406, -0.012167684, 0.021260925, -0.08139455, -0.010093985, 0.007983544, -0.032528505, -0.015999917, -0.01380362, 0.001309587, -0.0014520073, -0.04662374, -0.011653038, 0.008910889, -0.0087932, 0.043146037, -0.023925018, -0.024911307, -0.037911803, -0.019784097, 0.027514134, 0.035158217, 0.0016353147, 0.06662111, 0.06200062, -0.043098696, 0.025108913, -0.0036388384, 0.06255261, -0.0072050323, -0.03019539, -0.022023236, 0.037318517, -0.011165825, -0.00053298415, 0.010755897, -0.053007048, 0.021053808, 0.033987153, -0.022242764, -0.005185816, -0.06184692, -0.055027723, 0.029293697, 0.043532226, 0.015209735, -0.02396273, -0.00702

Building an embeddings database

In [ ]:

DOCUMENT1 = {
    "title": " General promptin / zero prompt",
    "content": """A zero-shot5 prompt is the simplest type of prompt. It only provides a description of a task and some text for the LLM to get started with.
This input could be anything: a question, a start of a story, or instructions. The name zero-shot stands for ’no examples Let’s use Vertex AI Studio (for Language) in Vertex AI,6 which provides a playground to test
prompts. In Table 1, you will see an example zero-shot prompt to classify movie reviews.
The table format as used below is a great way of documenting prompts. Your prompts will
likely go through many iterations before they end up in a codebase, so it’s important to keep
track of your prompt engineering work in a disciplined, structured way. More on this table
format, the importance of tracking prompt engineering work, and the prompt development
process is in the Best Practices section later in this chapter (“Document the various prompt
attempts”).
The model temperature should be set to a low number, since no creativity is needed, and we
use the gemini-pro default top-K and top-P values, which effectively disable both settings
(see ‘LLM Output Configuration’ above). Pay attention to the generated output. The words
disturbing and masterpiece should make the prediction a little more complicated, as both
words are used in the same sentence"""}

DOCUMENT2 = {
    "title" : "System prompting",
    "content" : """ System prompts can be useful for generating output that meets specific requirements. The
name ‘system prompt’ actually stands for ‘providing an additional task to the system’. For
example, you could use a system prompt to generate a code snippet that is compatible with
a specific programming language, or you could use a system prompt to return a certain
structure. Have a look into Table 4, where I return the output in JSON format.
There are some benefits in returning JSON objects from a prompt that extracts data. In
a real-world application I don’t need to manually create this JSON format, I can already
return the data in a sorted order (very handy when working with datetime objects), but most
importantly, by prompting for a JSON format it forces the model to create a structure and
limit hallucinations.
 System prompts can also be really useful for safety and toxicity. To control the output, simply
add an additional line to your prompt like: ‘You should be respectful in your answer.’."""}


DOCUMENT3 = {
    "title":  "Step-back prompting",
    "content": """Step-back8 prompting is a technique for improving the performance by prompting the LLM
to first consider a general question related to the specific task at hand, and then feeding the
answer to that general question into a subsequent prompt for the specific task. This ‘step
back’ allows the LLM to activate relevant background knowledge and reasoning processes
before attempting to solve the specific problem.
 By considering the broader and underlying principles, LLMs can generate more accurate and
insightful responses. Step-back prompting encourages LLMs to think critically and apply their
knowledge in new and creative ways. It changes the final prompt doing the task by utilizing
more knowledge in the LLM’s parameters than would otherwise come into play when the LLM
is prompted directly.
 It can help to mitigate biases in LLM responses, by focusing on general principles instead of
specific details, step-back prompting.
 Let’s have a look into these examples to understand how step-back prompting can improve
the results. Let’s first review a traditional prompt (Table 8) before we compare it to a step
back prompt (Table 9)."""}

DOCUMENTS = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


i will Organize the contents of the dictionary into a dataframe for better visualization.



In [ ]:
df = pd.DataFrame(DOCUMENTS)


In [ ]:
df.columns = ['Title', 'Text']


In [ ]:
df

,Title,Text
0,General promptin / zero prompt,A zero-shot5 prompt is the simplest type of pr...
1,System prompting,System prompts can be useful for generating o...
2,Step-back prompting,Step-back8 prompting is a technique for improv...


In [ ]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  response = client.models.embed_content(
        model=EMBEDDING_MODEL_ID,
        contents=text,
        config=types.EmbedContentConfig(
            task_type="retrieval_document",
            title=title
        )
    )

  return response.embeddings[0].values

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,General promptin / zero prompt,A zero-shot5 prompt is the simplest type of pr...,"[-0.041577224, -0.07019523, -0.023856534, -0.0..."
1,System prompting,System prompts can be useful for generating o...,"[0.012838747, -0.080515735, -0.033744026, -0.0..."
2,Step-back prompting,Step-back8 prompting is a technique for improv...,"[0.003742131, -0.049119502, -0.039598748, 0.01..."


In [ ]:
import numpy as np
import textwrap


MODEL_ID = "gemini-2.0-flash"

def find_passage(query, dataframe, threshold=0.5):
    """
    Compute the distances between the query and each document in the dataframe
    using the dot product.
    """
    query_embedding = client.models.embed_content(
        model=EMBEDDING_MODEL_ID,
        contents=query,
        config=types.EmbedContentConfig(
            task_type="retrieval_document",
        )
    )

    dot_products = np.dot(
        np.stack(dataframe['Embeddings']),
        query_embedding.embeddings[0].values
    )

    idx = np.argmax(dot_products)
    if dot_products[idx] < threshold:  # Check similarity threshold
        return "I'm sorry, I couldn't find a relevant passage in the documents."
    else:
        return dataframe.iloc[idx]['Text']





In [ ]:
def make_prompt(query, relevant_passage):
    # Check if relevant_passage indicates no passage was found
    if relevant_passage == "I'm sorry, I couldn't find a relevant passage in the documents.":
        return f"QUESTION: {query}\n\nI'm sorry, I don't have enough information to answer this question based on the provided documents."

    # If a passage was found, proceed with normal prompt creation
    escaped = (
        relevant_passage
        .replace("'", "")
        .replace('"', "")
        .replace("\n", " ")
    )
    prompt = textwrap.dedent(f"""
        You are a helpful and informative bot that answers questions using text
        from the reference passage included below. Be sure to respond in a
        complete sentence, being comprehensive, including all relevant
        background information.

        However, you are talking to a non-technical audience, so be sure to
        break down complicated concepts and strike a friendly and conversational
        tone. If the passage is irrelevant to the answer, you may ignore it.

        QUESTION: '{query}'
        PASSAGE: '{escaped}'

        ANSWER:
    """)

    return prompt

In [ ]:
def ask_questions(dataframe):
    while True:
        query = input("Ask a question (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break

        passage = find_passage(query, dataframe)
        prompt = make_prompt(query, passage)

        # Generate answer using client.models.generate_content
        answer = client.models.generate_content(
            model=MODEL_ID,  # Assuming MODEL_ID is defined earlier
            contents=prompt,
        )

        display(Markdown(answer.text))  # Display the answer as markdown

# Start the interactive question-answering loop
ask_questions(df)

Ask a question (or type 'exit' to quit): what is step back prompt 


Step-back prompting is a clever technique that helps large language models (LLMs) perform better by first asking them to think about a general question related to the specific task they're facing, and then using that general answer to help them with the specific task. Basically, it's like encouraging the LLM to take a "step back" and consider the bigger picture before diving into the details, allowing the LLM to use more information to solve the task.


Ask a question (or type 'exit' to quit): exit
